<a href="https://colab.research.google.com/github/datafyresearcher/datafy-finetuning-beginner/blob/main/notebooks/Basic/03_Training_lab_student_lamini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#===> Run this block, when using the Google Colab. Otherwise, do not run it.

if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  # Install the package
  ! pip install datasets jsonlines python-dotenv lamini transformers[torch] -q
else:
  print('Not running on CoLab')

# Training

## Technically, it's only a few lines of code to run on GPUs (elsewhere, ie. on Lamini).
```
from llama import BasicModelRunner

model = BasicModelRunner("EleutherAI/pythia-410m")
model.load_data_from_jsonlines("lamini_docs.jsonl")
model.train()
```
1. Choose base model.
2. Load data.
3. Train it. Returns a model ID, dashboard, and playground interface.

### Let's look under the hood at the core code running this! This is the open core of Lamini's `llama` library :)

In [6]:
import datasets
import logging
import torch

from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner

logger = logging.getLogger(__name__)
global_config = None

### Load the Lamini docs dataset

In [ ]:
dataset_name = "lamini_docs.jsonl"
dataset_path = f"/content/{dataset_name}"
use_hf = False

In [ ]:
dataset_path = "lamini/lamini_docs"
use_hf = True

### Set up the model, training config, and tokenizer

In [ ]:
model_name = "EleutherAI/pythia-70m"

In [ ]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)

print(train_dataset)
print(test_dataset)

### Load the base model

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

In [ ]:
base_model.to(device)

### Define function to carry out inference

In [ ]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

### Try the base model

In [ ]:
test_text = test_dataset[0]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {test_dataset[0]['answer']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

### Setup training

In [ ]:
max_steps = 3

In [ ]:
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

In [ ]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [ ]:
model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

In [ ]:
trainer = Trainer(
    model=base_model,
    model_flops=model_flops,
    total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

### Train a few steps

In [ ]:
training_output = trainer.train()

### Save model locally

In [ ]:
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

In [ ]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)


In [ ]:
finetuned_slightly_model.to(device)


### Run slightly trained model

In [ ]:
test_question = test_dataset[0]['question']
print("Question input (test):", test_question)

print("Finetuned slightly model's answer: ")
print(inference(test_question, finetuned_slightly_model, tokenizer))

In [ ]:
test_answer = test_dataset[0]['answer']
print("Target answer output (test):", test_answer)

### Run same model trained for two epochs

In [ ]:
finetuned_longer_model = AutoModelForCausalLM.from_pretrained("lamini/lamini_docs_finetuned")
tokenizer = AutoTokenizer.from_pretrained("lamini/lamini_docs_finetuned")

finetuned_longer_model.to(device)
print("Finetuned longer model's answer: ")
print(inference(test_question, finetuned_longer_model, tokenizer))

### Run much larger trained model and explore moderation

In [ ]:
# LAMINI_API_KEY is saved in `.env` file

# !export LAMINI_API_KEY= "<YOUR-KEY-HERE>"
from dotenv import load_dotenv
import os
load_dotenv()
LAMINI_API_KEY=os.getenv("LAMINI_API_KEY")

from llama import LLMEngine

llm = LLMEngine(
    id="marketing",
    config={
        "production": {
            "key": LAMINI_API_KEY,
        }
    },
)

In [ ]:
bigger_finetuned_model = BasicModelRunner(model_name_to_id["bigger_model_name"])
bigger_finetuned_output = bigger_finetuned_model(test_question)
print("Bigger (2.8B) finetuned model (test): ", bigger_finetuned_output)

In [ ]:
count = 0
for i in range(len(train_dataset)):
 if "keep the discussion relevant to Lamini" in train_dataset[i]["answer"]:
  print(i, train_dataset[i]["question"], train_dataset[i]["answer"])
  count += 1
print(count)

### Explore moderation using small model
First, try the non-finetuned base model:

In [ ]:
base_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
base_model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")
print(inference("What do you think of Mars?", base_model, base_tokenizer))

### Now try moderation with finetuned small model

In [ ]:
print(inference("What do you think of Mars?", finetuned_longer_model, tokenizer))

### Finetune a model in 3 lines of code using Lamini

In [ ]:
model = BasicModelRunner("EleutherAI/pythia-410m")
model.load_data_from_jsonlines("lamini_docs.jsonl")
model.train(is_public=True) # -> returns an ID, dashboard, and chat interface

In [34]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!git status

Refresh index: 100% (19/19), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   notebooks/01_Why_finetuning_lab_student.ipynb
	modified:   notebooks/02_Where_finetuning_fits_in_lab_student.ipynb
	modified:   notebooks/03_Instruction_tuning_lab_student.ipynb
	modified:   notebooks/04_Data_preparation_lab_student.ipynb
	modified:   notebooks/05_Training_lab_student.ipynb
	modified:   notebooks/0_Preparing_setup_datafy_finetuning.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	alpaca_processed.jsonl
	lamini_docs_3_steps/
	lamini_docs_processed.jsonl

no changes added to commit (use "git add" and/or "git commit -a")


In [35]:
! git commit -m "Added Code to run in Google Colab"

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@c3678bff7eee.(none)')


Follow the link above. The training can take a few minutes. You may need to refresh the URL to detect the change from 'In Progress' to 'Completed'

![image.png](attachment:5b0e47de-93a9-4f8e-8cd7-a42de1ce4fe3.png)

![image.png](attachment:02381ae5-d062-4f87-b7fe-6ac45b7f4fdb.png)